In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime
from pathlib import Path 

In [ ]:
# Change the value of fileLoc to match where you saved the file.
fileLoc = "C:/Users/Elissa/Documents/DSMS/Capstone_SPL/Data/RawData/FULL_Checkouts_By_Title__Physical_Items_.csv"

In [ ]:
with open(fileLoc) as ckobpi_file:
    
    header = ckobpi_file.readline()
    print(header)
        
    
    date_dict = {}
    itemType_dict = {}
    sub_dict = {}

    
    line_num=1
    twenty19_21_num = 0
    

    while line_num < 120000000:
        
        # declare variables
        sub_dict = {}
        
        # We use readline() to read in the next line / title
        try:
            cko_item = ckobpi_file.readline()
        except:
            continue

        #break out of the loop if we hit the end of the file
        if not cko_item:
            break

        cko_item_list = cko_item.split(",")
        
        try:
            if cko_item_list[1] == "2018" or cko_item_list[1] == "2019" or cko_item_list[1] == "2020" or cko_item_list[1] == "2021" :

                twenty19_21_num += 1
                
                itemType = cko_item_list[4]

                try:
                    cko_date, cko_time, cko_ampm = cko_item_list[-1].split(" ")
                except:
                    continue


                try:
                    cko_mo, cko_daynum, cko_year = cko_date.split("/")
                except:
                    continue
                    

                if(cko_date in date_dict):  #if date in dictionary add count
                    date_dict[cko_date] = date_dict[cko_date]+1
                else:
                    date_dict[cko_date]= 1

                
                if cko_date in itemType_dict:  #if date in dictionary check itemType AND increment 
                    if itemType in itemType_dict[cko_date]:

                        itemType_dict[cko_date][itemType] = itemType_dict[cko_date][itemType] +1
                    else:
                        sub_dict[itemType] = 1
                        #print(sub_dict)
                        itemType_dict[cko_date][itemType] = 1
                else:
                    sub_dict[itemType] = 1
                    itemType_dict[cko_date] = sub_dict      
                
                

        except:
            continue
            
        
            


        line_num += 1

        if line_num % 5000000 == 0:
            print(line_num, twenty19_21_num)
            
    print("Done: ", line_num)


In [ ]:
dateCirc_df = pd.DataFrame.from_dict(date_dict, orient='index')
dateCirc_df.columns =['totCirc']
dateCirc_df.index.name = 'date'
dateCirc_df.reset_index(inplace=True)

typeCirc_df = pd.DataFrame.from_dict(itemType_dict, orient='index')
typeCirc_df.index.name = 'date'
typeCirc_df.reset_index(inplace=True)

In [ ]:
dateCirc_df.head()

In [ ]:
typeCirc_df.head()

In [ ]:
# write individual file
filepath = Path('C:/Users/Elissa/Documents/DSMS/Capstone_SPL/Data/dateCirc_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
dateCirc_df.to_csv(filepath,index=True) 

In [ ]:
# write individual file
filepath = Path('C:/Users/Elissa/Documents/DSMS/Capstone_SPL/Data/typeCirc_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
typeCirc_df.to_csv(filepath,index=True) 

In [ ]:
# join data frames
allCirc_df = pd.merge(dateCirc_df, typeCirc_df, how="outer", on="date")

In [ ]:
allCirc_df.head()

In [ ]:
# create list of dates as strings, and lists weekday as 1-7 where 1 is Monday
datelist = []
daylist = []

daterange = pd.date_range(start="2018-01-01",end="2021-12-31")

for x in range(len(daterange)):
    datelist.append(daterange.date[x].strftime("%m/%d/%Y"))
    daylist.append(daterange.date[x].isoweekday())

In [ ]:
data = {'date':datelist, 'dayOfWeek':daylist}
weekDay_df = pd.DataFrame(data)

In [ ]:
weekDay_df.loc[weekDay_df['dayOfWeek']==1,'nameOfDay'] = 'Monday'
weekDay_df.loc[weekDay_df['dayOfWeek']==2,'nameOfDay'] = 'Tuesday'
weekDay_df.loc[weekDay_df['dayOfWeek']==3,'nameOfDay'] = 'Wednesday'
weekDay_df.loc[weekDay_df['dayOfWeek']==4,'nameOfDay'] = 'Thursday'
weekDay_df.loc[weekDay_df['dayOfWeek']==5,'nameOfDay'] = 'Friday'
weekDay_df.loc[weekDay_df['dayOfWeek']==6,'nameOfDay'] = 'Saturday'
weekDay_df.loc[weekDay_df['dayOfWeek']==7,'nameOfDay'] = 'Sunday'

In [ ]:
weekDay_df.head()

In [ ]:
allCircDOW_df = pd.merge(allCirc_df, weekDay_df, how="outer", on="date")

In [ ]:
allCircDOW_df.head()

In [ ]:
#write to file

# write individual file
filepath = Path('C:/Users/Elissa/Documents/DSMS/Capstone_SPL/Data/allCircDOW_18-21_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
allCircDOW_df.to_csv(filepath,index=True) 